In [3]:
import pandas as pd

In [4]:
df_train = pd.read_csv('new_train_labels_v1.csv')
df_val = pd.read_csv('new_val_labels_v1.csv')
df_test = pd.read_csv('new_test_labels_v1.csv')

df = pd.concat([df_train, df_val, df_test])

df = df.reset_index(drop=True)

assert len(df) == len(df_train) + len(df_val) + len(df_test)

In [6]:
df.head()

,0,1,2,3,4,paths
0,0,0,0,0,0,chips/cloudfree-merge-polygons/dataset_v2/misc...
1,0,0,0,0,0,chips/cloudfree-merge-polygons/dataset_v2/misc...
2,0,0,0,0,0,chips/cloudfree-merge-polygons/dataset_v2/misc...
3,0,0,0,0,0,chips/cloudfree-merge-polygons/dataset_v2/misc...
4,0,0,0,0,0,chips/cloudfree-merge-polygons/dataset_v2/misc...


In [8]:
df.loc[0, 'paths'].split('/')

['chips',
 'cloudfree-merge-polygons',
 'dataset_v2',
 'misc',
 '100',
 '12',
 '12_2200_4700.tif']

In [14]:
def get_poly_paths(df, polygon_id=91):
    df2 = df.copy()
    df2['polygon_id'] = df2['paths'].apply(lambda x: int(x.split('/')[5]))
    df_poly = df2[df2['polygon_id'] == polygon_id]
    return list(set(df_poly['paths'].tolist()))

In [15]:
poly_paths = get_poly_paths(df)

len(poly_paths)

1160

In [16]:
poly_paths[0:5]

['chips/cloudfree-merge-polygons/dataset_v2/misc/100/91/91_2300_100.tif',
 'chips/cloudfree-merge-polygons/dataset_v2/misc/100/91/91_800_2200.tif',
 'chips/cloudfree-merge-polygons/dataset_v2/misc/100/91/91_1000_2400.tif',
 'chips/cloudfree-merge-polygons/dataset_v2/misc/100/91/91_2800_3000.tif',
 'chips/cloudfree-merge-polygons/dataset_v2/misc/100/91/91_900_2300.tif']

In [17]:
isl_paths = [path for path in poly_paths if path.split('/')[3] == 'ISL']
len(isl_paths)

75

In [27]:
isl_paths[0]

'chips/cloudfree-merge-polygons/dataset_v2/ISL/100/91/91_1600_1900.tif'

In [25]:
import boto3


def download_chip(chip_url, dest_dir):
    if dest_dir[-1] != '/':
        dest_dir = dest_dir + '/'
        
    #print(dest_dir)
    
    s3 = boto3.resource('s3')
    bucket = chip_url.split('/')[2]
    chip_key = '/'.join(chip_url.split('/')[3:])
    chip_filename = chip_url.split('/')[-1]
    dest_path = dest_dir + chip_filename
    #print(chip_key)
    #print(dest_path)
    s3.Bucket(bucket).download_file(chip_key, dest_path)

In [28]:
dest_dir = 'D:/canopy_data/error_fixing/chips'

for i, path in enumerate(isl_paths[1:], 1):
    print(f'Downloading chip {i} of {len(isl_paths[1:])}', end='\r', flush=True)
    chip_url = 's3://canopy-production-ml/' + path
    download_chip(chip_url, dest_dir)

In [29]:
def download_model(model_url,weights_url,model_filename="D:/canopy_data/error_fixing/model.h5",
                   model_weights_filename="D:/canopy_data/error_fixing/model_weights.h5"):

    s3 = boto3.resource('s3')

    #Download Model, Weights

    bucket = model_url.split("/")[2]
    model_key = "/".join(model_url.split("/")[3:])
    s3.Bucket(bucket).download_file(model_key, model_filename)
    weights_key = "/".join(weights_url.split("/")[3:])
    s3.Bucket(bucket).download_file(weights_key, model_weights_filename)

    return 

In [30]:
model_url = "s3://canopy-production-ml/inference/model_files/model-best.h5"
weights_url = "s3://canopy-production-ml/inference/model_files/model_weights_best.h5"

download_model(model_url,weights_url)

In [8]:
l = ['train', 'val', 'test']

for i, df in enumerate([df_train, df_val, df_test]):
    df_filter = df[(df['1'] == 1) & (df['paths'].str.contains('/91/'))]
    if len(df_filter) > 0:
        print('Success for', l[i])

Success for test
